In [1]:
!pip install pinecone

In [1]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_EGSeY_HW3vERNzeBG2VKLw3eNDNbpjupSH8ABDk8Edgh4aaehjtptKMekB8br7fN2cpXm")
index = pc.Index("cyber")

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load all 3 PDFs (use raw strings for all Windows paths)
pdf_paths = [
    r"C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\CANVAS-Book-Chapter-Herrmann-Pridhl.pdf",
    r"C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\CYBER SECURITY ESSENTIALS Digital Notes.pdf",
    r"C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\Cyber Security.pdf"
]

all_docs = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    all_docs.extend(loader.load())

# Chunk the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)


In [10]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")  # 768-dim

# OR if you really want 1024-dim (rare), try:
embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large")  # ~1024-dim


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\student\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\student\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/e5-large")
print(model.get_sentence_embedding_dimension())  # should show 1024


1024


In [7]:
import os

os.environ["PINECONE_API_KEY"] = "pcsk_EGSeY_HW3vERNzeBG2VKLw3eNDNbpjupSH8ABDk8Edgh4aaehjtptKMekB8br7fN2cpXm"


In [12]:
# This is correct
vectorstore = LangchainPinecone.from_documents(
    documents=chunks,
    embedding=embedding,
    index_name="cyber",  # pass the index *name* as a string
    namespace="cyber-namespace"
)


In [15]:
from langchain.vectorstores import FAISS


In [ ]:
#pcsk_6kqjxL_9oBp1EzhJ9xABmnwRwdZtJwhSR497mfPNLNj62TqDdfnCqeDuY5t8MQuKo3E4tX
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from pinecone import Pinecone

# -------- 1. Load 1024-D Embedding Model --------
embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large")  # 1024-dim ✅

# -------- 2. Connect to Pinecone --------
pc = Pinecone(api_key="pcsk_EGSeY_HW3vERNzeBG2VKLw3eNDNbpjupSH8ABDk8Edgh4aaehjtptKMekB8br7fN2cpXm")  # Replace with your actual key

index_name = "cyber"
namespace = "cyber-namespace"

vectorstore = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embedding,
    namespace=namespace
)

# -------- 3. Load a Local LLM (No Login Required) --------
model_id = "google/flan-t5-base"  # Lightweight and fast for CPUs

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.3
)

llm = HuggingFacePipeline(pipeline=pipe)

# -------- 4. Build the RAG Pipeline --------


# -------- 5. Ask a Question --------


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer:
 Man-in-the-Middle (MitM) Attacks

--- Sources ---
C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\Cyber Security.pdf
C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\CYBER SECURITY ESSENTIALS Digital Notes.pdf
C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\Cyber Security.pdf
C:\Users\student\Desktop\314\Cyber Security Project\LLM Attacks\llmdata\CYBER SECURITY ESSENTIALS Digital Notes.pdf


In [25]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)


In [27]:
query = "what are the types of denial of services?"
result = qa_chain({"query": query})

print("Answer:", result['result'])

# Print top 3 retrieved docs
for i, doc in enumerate(result['source_documents'], 1):
    print(f"--- Source Document {i} ---")
    print(doc.page_content)
    print()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer: Denial of Service (DOS) Attack DoS attacks work by flooding systems, servers, and/or networks with traffic to overload resources and bandwidth. The result is rendering the system unable to process and fulfill legitimate requests. In addition to denial-of-service (DoS) attacks, there are also distributed denial-of-service (DDoS) attacks. DoS attacks saturate a system’s resources with the goal of impeding response to service requests. On the other hand, a DDoS attack is launched from several infected host machines with the goal of achieving service denial and taking a system offline, thus paving the way for another attack to enter the network/environment. The most common types of DoS and DDoS attacks are the TCP SYN flood attack, teardrop attack, smurf attack, ping-of-death attack, and botnets.
--- Source Document 1 ---
Denial-of-Service (DOS) Attack 
DoS attacks work by flooding systems, servers, and/or networks with traffic to overload resources and 
bandwidth. The result is re

In [28]:
!pip uninstall pinecone-client -y


Found existing installation: pinecone-client 3.0.0
Uninstalling pinecone-client-3.0.0:
  Successfully uninstalled pinecone-client-3.0.0


In [29]:
pip install pinecone


Note: you may need to restart the kernel to use updated packages.


In [30]:
!pip uninstall pinecone-client pinecone -y


Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0


In [31]:
!pip install pinecone


  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


In [34]:
!pip uninstall pinecone-client


In [35]:
!python -m pip install pinecone
